In [1]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)
import os

import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from callbacks import all_callbacks

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
import tensorflow.compat.v1 as tf1

2022-10-29 19:57:23.072432: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-29 19:57:23.285634: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-29 19:57:23.291843: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:
2022-10-29 19:57:23.291859: I tensorflow/stream_executor/cuda/cudart_stub.c

## Fetch the cardio dataset

In [2]:
df = pd.read_csv('./dataset.csv', sep = ';')
print (np.shape(df))

(2126, 22)


,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X13,X14,X15,X16,X17,X18,X19,X20,X21,Y
0,120,0.000,0.000,0.000,0.000,0.0,0.0,73,0.5,43,...,62,126,2,0,120,137,121,73,1,2
1,132,0.006,0.000,0.006,0.003,0.0,0.0,17,2.1,0,...,68,198,6,1,141,136,140,12,0,1
2,133,0.003,0.000,0.008,0.003,0.0,0.0,16,2.1,0,...,68,198,5,1,141,135,138,13,0,1
3,134,0.003,0.000,0.008,0.003,0.0,0.0,16,2.4,0,...,53,170,11,0,137,134,137,13,1,1
4,132,0.007,0.000,0.008,0.000,0.0,0.0,16,2.4,0,...,53,170,9,0,137,136,138,11,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,140,0.000,0.000,0.007,0.000,0.0,0.0,79,0.2,25,...,137,177,4,0,153,150,152,2,0,2
2122,140,0.001,0.000,0.007,0.000,0.0,0.0,78,0.4,22,...,103,169,6,0,152,148,151,3,1,2
2123,140,0.001,0.000,0.007,0.000,0.0,0.0,79,0.4,20,...,103,170,5,0,153,148,152,4,1,2
2124,140,0.001,0.000,0.006,0.000,0.0,0.0,78,0.4,27,...,103,169,6,0,152,147,151,4,1,2


In [3]:
X = df.drop('Y', axis = 1).values

0       2
1       1
2       1
3       1
4       1
       ..
2121    2
2122    2
2123    2
2124    2
2125    1
Name: Y, Length: 2126, dtype: int64


In [5]:
le = LabelEncoder()
y = le.fit_transform(y)
print(le.classes_)
y = to_categorical(y, 3)

0       2
1       1
2       1
3       1
4       1
       ..
2121    2
2122    2
2123    2
2124    2
2125    1
Name: Y, Length: 2126, dtype: int64
[1 2 3]
[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [10]:
ls=np.argmax(Y_test,axis=1)+1

[1 1 1 1 1 3 1 3 1 2 1 3 1 1 1 1 1 1 2 1 1 1 3 1 1 3 3 1 1 1 1 1 1 2 1 1 1
 3 2 2 1 2 1 1 1 1 1 3 1 1 1 1 2 2 2 1 1 1 2 1 2 1 1 1 1 1 2 1 1 1 1 1 1 1
 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 3 2 1 2 1 1 1 1 2 1 1 3 1 2
 2 1 2 1 3 1 1 2 1 1 2 1 1 3 3 3 1 1 1 3 1 1 1 1 1 1 1 1 2 2 1 1 1 3 1 1 1
 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 2 1 1 1 1 2 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 3
 1 1 1 3 2 1 1 1 2 1 2 1 1 1 2 1 1 1 2 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2
 1 1 1 1 2 1 1 1 2 1 1 1 2 2 1 1 1 1 1 1 1 1 1 3 1 1 3 1 1 1 1 1 1 1 1 1 1
 2 2 2 1 1 2 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 2 3 1 3 1 1 1 1 1 1 1 2 1 1 2 1 1 1 1
 1 1 2 1 1 1 1 1 2 3 3 1 1 1 3 2 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1
 1 1 1 1 1 3 1 1 1 3 1 1 1 1 1 1 2 1 2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 3 1 1 2
 1 1 1 2 1 1 1 1 1 1 2 1 1 1 2 1 1 1 2 1 2 1 1 1 1 1 1 1 1 3 1 1 3 1 1 2 1
 2 1 1 1 1 1 1 1 2 1 2 3 2 1 1 1 1 1 3 1 1 1 1 2 1 2 1 1 1 2 1 1 1 2 1 3 1
 1 1 1 1 1 1 2 1 1 1 1 1 

In [12]:
print("Shape of X_train: ",X_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_train: ",Y_train.shape)
print("Shape of y_test",Y_test.shape)

Shape of X_train:  (1488, 21)
Shape of X_test:  (638, 21)
Shape of y_train:  (1488, 3)
Shape of y_test (638, 3)


In [14]:
import pickle
from joblib import load
loaded_model = load('./Cardio.MLP_clf.joblib')

/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.24.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.24.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [18]:
w1=loaded_model.coefs_[0]
b1=loaded_model.intercepts_[0]
w2=loaded_model.coefs_[1]
b2=loaded_model.intercepts_[1]

In [19]:
wb1=[]
wb1.append(w1)
wb1.append(b1)

wb2=[]
wb2.append(w2)
wb2.append(b2)

### genetic algorithm to determine the relu size, weight size, bias size and sparsity

In [25]:
import blackbox as bb
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.problem import Problem
from pymoo.operators.repair.rounding import RoundingRepair
import area as ar
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

import tensorflow as tf
import os
import pandas as pd
import seaborn as sb

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from callbacks import all_callbacks
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
import tensorflow.compat.v1 as tf1
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from qkeras.utils import model_save_quantized_weights
from sklearn.preprocessing import MinMaxScaler
import hls4ml
import write_mlp_mergemult_ps as wv

/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/hls4ml/converters/__init__.py:16: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!")


In [26]:
with open("paretos_cardio", "rb") as fp:   # Unpickling
    paretos = pickle.load(fp)
with open("costs_cardio", "rb") as fp:   # Unpickling
    costs = pickle.load(fp)

### genetic algorithm to determine the relu size, weight size, bias size and sparsity

In [ ]:
class MyProblem(ElementwiseProblem):

    def __init__(self,weightsbiases1,weightsbiases2,layer1,layer2,layer3,X_test,X_train,Y_test,Y_train):
        self.weightsbiases1=weightsbiases1
        self.weightsbiases2=weightsbiases2
        self.layer1=layer1
        self.layer2=layer2
        self.layer3=layer3
        self.X_test=X_test
        self.X_train=X_train
        self.Y_test=Y_test
        self.Y_train=Y_train
        #x[0]: relu_size
        #x[1]: weight size layer1
        #x[2]: bias size layer1
        #x[3]: weight size layer2
        #x[4]: bias size layer2
        #x[5]: pruning sparsity
        #x[6]: input size
        #x[7]: int relu size
        
        super().__init__(n_var=8,
                         n_obj=2,
                         n_ieq_constr=0,
                         xl=np.array([3,2,2,2,2,0,3,0]),
                         xu=np.array([6,7,5,7,5,0,4,1]),
                         vtype=int)

    def _evaluate(self, x, out, *args, **kwargs):
        accuracy, weights = bb.blackbox(self.weightsbiases1,self.weightsbiases2, x[0], x[1], x[2], x[3], x[4], x[5], x[6] , x[7],  self.layer1, self.layer2, self.layer3, self.X_test, self.Y_test, self.X_train, self.Y_train)
        f1 = 1- accuracy
        f2 = ar.area(weights,x[6],x[0],x[1],x[3],self.layer1,self.layer2,self.layer3)

        out["F"] = [f1, f2]


In [ ]:
layer1=21
layer2=3
layer3=3
problem = MyProblem(wb1,wb2,layer1,layer2,layer3,X_test,X_train,Y_test,Y_train)

In [ ]:
algorithm = NSGA2(
    pop_size=50,
    n_offsprings=40,
    sampling=IntegerRandomSampling(),
    crossover=SBX(vtype=float, repair=RoundingRepair()),
    mutation=PM( vtype=float, repair=RoundingRepair()),
    eliminate_duplicates=True
)

In [ ]:
from pymoo.termination import get_termination

termination = get_termination("n_gen",50)

In [ ]:
%%time
from pymoo.optimize import minimize

res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)
X = res.X
F = res.F

In [ ]:
print(res.F)

In [ ]:
print(res.X)

### plot all the solutions from the final generation

In [ ]:
from pymoo.visualization.scatter import Scatter
pop=res.pop
vals=pop.get("F")
plot = Scatter()
plot.add(problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
plot.add(vals, facecolor="none", edgecolor="red")
plot.show()

### plot the pareto solutions from the final generation

In [ ]:
plot = Scatter()
plot.add(problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
plot.add(res.F, facecolor="none", edgecolor="blue")
plot.show()

### define an optimal solution

In [ ]:
paretos=res.X
costs=res.F

In [ ]:
# import pickle
# with open("paretos_cardio", "wb") as fp:   #Pickling
#     pickle.dump(paretos, fp)
# with open("costs_cardio", "wb") as fp:   #Pickling
#     pickle.dump(costs, fp)

In [ ]:
with open("paretos_cardio", "rb") as fp:   # Unpickling
    paretos = pickle.load(fp)
with open("costs_cardio", "rb") as fp:   # Unpickling
    costs = pickle.load(fp)

In [ ]:
layer_1=21
layer_2=3
layer_3=3
from importlib import reload

In [ ]:
import pareto_verilogs as pv
import pareto_clustering_verilogs as pcv
import only_pruning_verilog as opv

i=0
layer1=21
layer2=3
layer3=3
epochs=5
lr=0.003
#check the only pruning scenario
opv.generate(i,layer1,layer2,layer3,lr,epochs,X_train,Y_train,X_test,Y_test,wb1,wb2)


In [ ]:
reload(pv)
reload(opv)
reload(pcv)
reload(pwsv)
reload(poqv)

In [ ]:
for solution in paretos:
    relusize_f=solution[0]
    weight_size_f1=solution[1]
    bias_size_f1=solution[2]
    weight_size_f2=solution[3]
    bias_size_f2=solution[4]
    sparsity=solution[5]
    inputsize=solution[6]
    relusize_int=solution[7]
    model=pv.generate(i,relusize_f,weight_size_f1,bias_size_f1,weight_size_f2,bias_size_f2,sparsity,inputsize,relusize_int,layer1,layer2,layer3,lr,epochs,X_train,Y_train,X_test,Y_test,wb1,wb2)
    pcv.generate(model,i,relusize_f,weight_size_f1,bias_size_f1,weight_size_f2,bias_size_f2,inputsize,relusize_int,layer1,layer2,layer3,X_train,Y_train,X_test,Y_test)
    i += 1

### custom weight sharing is not feasible, all pareto are hw friendly